In [1]:
# Import Required Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from IPython.display import display
import gradio as gr


In [2]:
# Set Paths
base_path = 'TrashType_Image_Dataset'  # Folder containing subfolders of classes

In [3]:
# Data Preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
train_gen = train_datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_gen = train_datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [4]:
# Build the Model with EfficientNetV2B2
input_shape = (224, 224, 3)

base_model = EfficientNetV2B2(
    weights='imagenet',
    include_top=False,
    input_tensor=Input(shape=input_shape)
)
base_model.trainable = False  # freeze for now

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# Train the model
model.fit(train_gen, epochs=13, validation_data=val_gen)


Epoch 1/13


C:\Users\anura\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6153 - loss: 1.0570 - val_accuracy: 0.8310 - val_loss: 0.4802
Epoch 2/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8734 - loss: 0.4048 - val_accuracy: 0.8569 - val_loss: 0.4417
Epoch 3/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 62s 972ms/step - accuracy: 0.8824 - loss: 0.3354 - val_accuracy: 0.8767 - val_loss: 0.4074
Epoch 4/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.9171 - loss: 0.2482 - val_accuracy: 0.8728 - val_loss: 0.4229
Epoch 5/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.9235 - loss: 0.2444 - val_accuracy: 0.8787 - val_loss: 0.4150
Epoch 6/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 61s 954ms/step - accuracy: 0.9382 - loss: 0.1980 - val_accuracy: 0.8767 - val_loss: 0.4269
Epoch 7/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 61s 958ms/step - accuracy: 0.9554 - loss: 0.1744 - val_accuracy: 0.8827 - val_loss: 0.4377
Epoch 8/13
64/64 ━━━━━━━━━━━━━━━━━━━━ 61s 960ms/step - accuracy: 0.9524 - loss: 0.1506 - val_accuracy: 0.8807 - val_loss

In [6]:
# Save the model
model.save("garbage_classifier_effnet.h5")
print("\n✅ Model trained and saved as garbage_classifier_effnet.h5")


✅ Model trained and saved as garbage_classifier_effnet.h5


In [7]:
# Load the trained model
model.save("efficientnet_garbage_classifier.h5")


In [8]:
# Define class labels (same order as training)
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [9]:
# Define prediction function
def predict_material(img):
    try:
        print("Image received")
        img = img.resize((224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        print("Preprocessing done")

        prediction = model.predict(img_array)
        print("Prediction done")

        predicted_class = class_labels[np.argmax(prediction)]
        confidence = np.max(prediction) * 100

        return f"📦 Material: {predicted_class}\n🎯 Accuracy: {confidence:.2f}%"
    except Exception as e:
        print("Error:", e)
        return f"❌ Error: {str(e)}"


In [10]:
# Create Gradio UI
import gradio as gr
interface = gr.Interface(
    fn=predict_material,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="♻️ Garbage Classification",
    description="Upload an image of garbage (plastic, paper, metal, etc.) to classify its type."
)

In [11]:
# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
